In [ ]:
import re, os
import glob
import datetime

import h5py
import pyart
import pyproj
import netCDF4
import gpmmatch
import numpy as np
import xarray as xr
import pandas as pd

import dask
import dask.bag as db
from dask.diagnostics import ProgressBar

In [ ]:
class NoPrecipitationError(Exception):
    pass

In [ ]:
def precip_in_domain(gpmset, grlon, grlat, rmax=150e3, rmin=20e3):
    georef = pyproj.Proj(f"+proj=aeqd +lon_0={grlon} +lat_0={grlat} +ellps=WGS84")
    gpmlat = gpmset.Latitude.values
    gpmlon = gpmset.Longitude.values

    xgpm, ygpm = georef(gpmlon, gpmlat)
    rproj_gpm = (xgpm ** 2 + ygpm ** 2) ** 0.5

    gr_domain = (rproj_gpm <= rmax) & (rproj_gpm >= rmin)
    if gr_domain.sum() < 10:
        info = f'The closest satellite measurement is {np.min(rproj_gpm / 1e3):0.4} km away from ground radar.'
        if gr_domain.sum() == 0:
            raise NoPrecipitationError('GPM swath does not go through the radar domain. ' + info)
        else:
            raise NoPrecipitationError('GPM swath is on the edge of the ground radar domain and there is not enough measurements inside it. ' + info)

    nprof = np.sum(gpmset.flagPrecip.values[gr_domain])
    if nprof < 10:
        raise NoPrecipitationError('No precipitation measured by GPM inside radar domain.')
        
    newset = gpmset.merge({'range_from_gr': (('nscan', 'nray'), rproj_gpm)})
    
    gpmtime0 = newset.nscan.where(newset.range_from_gr == newset.range_from_gr.min()).values.astype('datetime64[s]')
    gpmtime0 = gpmtime0[~np.isnat(gpmtime0)][0]
    
    del newset
    return nprof, gpmtime0

In [ ]:
def get_overpass_with_precip(gpmfile, radarset):
    gpmset = gpmmatch.io.read_GPM(gpmfile)
    data = dict()
    for n in range(len(radarset)):
        rid = radarset.id[n]
        rname = radarset.short_name[n]
        grlat = radarset.site_lat[n]
        grlon = radarset.site_lon[n]

        try:
            nprof, gpmtime = precip_in_domain(gpmset, grlat=grlat, grlon=grlon)
        except NoPrecipitationError:
            continue        

        data[rid] = (str(gpmtime),                     
                     rname,
                     str(grlon),
                     str(grlat),
                     str(nprof),
                     gpmfile)
    return data


In [ ]:
flist = sorted(glob.glob('/g/data/rq0/admin/calibration/sr_data/gpm_data/2020/08/**/*.*'))
# flist = sorted(glob.glob('/scratch/kl02/vhl548/gpm_data/**/*.*'))

In [ ]:
df = pd.read_csv('/home/548/vhl548/radar_site_list.csv')
ndf = df.drop_duplicates('id', keep='last').reset_index()
argslist = [(f, ndf) for f in flist]

In [ ]:
bag = db.from_sequence(argslist).starmap(get_overpass_with_precip)
with ProgressBar():
    rslt = bag.compute()

In [ ]:
for n in rslt:
    if len(n) == 0:
        continue
    for rid in n.keys():
        outpath = '/scratch/kl02/vhl548/gpm_output/overpass/'
        outfile = os.path.join(outpath, f'gpm.{rid:02}.csv')
        with open(outfile, 'a+') as fid:
            fid.write(','.join(n[rid]))
            fid.write('\n')